In [1]:
import tensorflow as tf
import numpy as np

In [2]:
labels = np.array([[1,1,1,0],
                   [1,1,1,0],
                   [1,1,1,0],
                   [1,1,1,0]], dtype=np.uint8)

predictions = np.array([[1,0,0,0],
                        [1,1,0,0],
                        [1,1,1,0],
                        [0,1,1,1]], dtype=np.uint8)

n_batches = len(labels)


In [3]:
labels

array([[1, 1, 1, 0],
       [1, 1, 1, 0],
       [1, 1, 1, 0],
       [1, 1, 1, 0]], dtype=uint8)

In [4]:
predictions

array([[1, 0, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 1, 0],
       [0, 1, 1, 1]], dtype=uint8)

## 精确率的计算公式
$$ Precision = \frac{truePositive}{truePositive + falsePositive} $$

In [5]:
# First,calculate precision over entire set of batches 
# using formula mentioned above
pred_p = (predictions > 0).sum()
# print(pred_p)
true_p = (labels*predictions > 0).sum()
# print(true_p)
precision = true_p / pred_p
print("Precision :%1.4f" %(precision))

Precision :0.8889


## 划重点

首先，这篇文章是受到[Ronny Restrepo](http://ronny.rest/blog/post_2017_09_11_tf_metrics/)的启发，
这是一篇很好的文章，将`tf.metrics.accuracy()`讲解滴很清楚，本文就模仿他的思路，验证一下`precision`的计算。

## 上述方法的问题

由于硬件方面的一些限制，导致此方法不能扩展到大型数据集，比如当数据集很大时，就无法一次性适应内存。
因而，为了使其可扩展，我们希望使评估指标能够逐步更新，每批新的预测和标签。 为此，我们需要跟踪两个值。

* 正确预测的正样本数量
* 预测样本中所有正样本的数量

In [6]:
# Initialize running variables
N_TRUE_P = 0
N_PRED_P = 0

In [7]:
# Specific steps
# Create running variables
N_TRUE_P = 0
N_PRED_P = 0

def reset_running_variables():
    """ Resets the previous values of running variables to zero """
    global N_TRUE_P, N_PRED_P
    N_TRUE_P = 0
    c = 0

def update_running_variables(labs, preds):
    global N_TRUE_P, N_PRED_P
    N_TRUE_P += ((labs * preds) > 0).sum()
    N_PRED_P += (preds > 0).sum()

def calculate_precision():
    global N_TRUE_P, N_PRED_P
    return float (N_TRUE_P) / N_PRED_P

## 怎么用上面的函数呢？

接下来的两个例子，给出了运用的具体代码，并且可以更好滴帮助我们理解`tf.metrics.precision()`的计算逻辑以及对应输出所代表的含义

### 样本整体准确率
* Overall precision

### 批次准确率
* Batch precision

In [8]:
# Overall precision
reset_running_variables()

for i in range(n_batches):
    update_running_variables(labs=labels[i], preds=predictions[i])

precision = calculate_precision()
print("[NP] SCORE: %1.4f" %precision)


[NP] SCORE: 0.8889


In [9]:
# Batch precision
for i in range(n_batches):
    reset_running_variables()
    update_running_variables(labs=labels[i], preds=predictions[i])
    prec = calculate_precision()
    print("- [NP] batch %d score: %1.4f" %(i, prec))

- [NP] batch 0 score: 0.1000
- [NP] batch 1 score: 0.1667
- [NP] batch 2 score: 0.2000
- [NP] batch 3 score: 0.1111


## 不要小瞧这两个变量和三个函数
上面说了这么多，感觉没有tensorflow的什么事哇，别急，先看一个tensorflow的官方文档

### 放一个官方的解释
[tensorflow/tensorflow/python/ops/metrics_impl.py](https://github.com/tensorflow/tensorflow/blob/r1.10/tensorflow/python/ops/metrics_impl.py)

* Github代码中precision的解释部分

> The `precision` function creates **two local variables**,
  `true_positives` and `false_positives`, that are used to compute the precision. This value is ultimately returned as `precision`, an idempotent operation that simply divides `true_positives` by the sum of `true_positives` and `false_positives`. 
  For estimation of the metric over a stream of data, the function creates an `update_op` operation that updates these variables and returns the `precision`. 
 
### 两个变量和 `tf.metrics.precision()`的关系
官方文档提及的**two local variables** ：`true_postives` 和 `false_positives`分别对应上文定义的两个变量。
* true_postives--N_TRUE_P
* false_postives--N_PRED_P - N_TRUE_P

### 三个函数和头大的`update_op`
官方文档提及的`update_op`和`precision`分别对应上文定义的两个函数
* precision--calculate_precision()
* update_op--update_running_variables()

大家不要被这个`update_op`搞晕，其实从字面来理解就是一个变量更新的操作，上文的代码中，就是通过`reset_running_variables()`的位置来决定何时对变量进行更新，其实就是对应于`tf.variables_initializer()`。我之所以一直用错这个API，是因为我将`tf.variables_initializer()`放在了错误的位置，导致变量没有按照我的预期正常更新，进而结果一直不正确。

In [10]:
# Overall precision using tensorflow
import tensorflow as tf

graph = tf.Graph()
with graph.as_default():
    # Placeholders to take in batches onf data
    tf_label = tf.placeholder(dtype=tf.int32, shape=[None])
    tf_prediction = tf.placeholder(dtype=tf.int32, shape=[None])

    # Define the metric and update operations
    tf_metric, tf_metric_update = tf.metrics.precision(tf_label,
                                                      tf_prediction,
                                                      name="my_metric")

    # Isolate the variables stored behind the scenes by the metric operation
    running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="my_metric")

    # Define initializer to initialize/reset running variables
    running_vars_initializer = tf.variables_initializer(var_list=running_vars)


with tf.Session(graph=graph) as session:
    session.run(tf.global_variables_initializer())

    # initialize/reset the running variables
    session.run(running_vars_initializer)

    for i in range(n_batches):
        # Update the running variables on new batch of samples
        feed_dict={tf_label: labels[i], tf_prediction: predictions[i]}
        session.run(tf_metric_update, feed_dict=feed_dict)

    # Calculate the score
    score = session.run(tf_metric)
    print("[TF] SCORE: %1.4f" %score)

[TF] SCORE: 0.8889


In [11]:
# Batch precision using tensorflow
with tf.Session(graph=graph) as session:
    session.run(tf.global_variables_initializer())

    for i in range(n_batches):
        # Reset the running variables
        session.run(running_vars_initializer)

        # Update the running variables on new batch of samples
        feed_dict={tf_label: labels[i], tf_prediction: predictions[i]}
        session.run(tf_metric_update, feed_dict=feed_dict)

        # Calculate the score on this batch
        score = session.run(tf_metric)
        print("[TF] batch %d score: %1.4f" %(i, score))


[TF] batch 0 score: 1.0000
[TF] batch 1 score: 1.0000
[TF] batch 2 score: 1.0000
[TF] batch 3 score: 0.6667


## 再次划重点
大噶一定要注意

`session.run(running_vars_initializer)`

`score = session.run(tf_metric)`

这两行代码在计算**整体样本精确度**以及**批次精确度**所在位置的不同。
澜子第一次的时候由于粗心，并没有注意两段代码的不同，才会导致**tf计算结果**和**普通计算结果**不一致

## 还需要注意的点

不要在一个`sess.run()`里面同时调用`tf_metric`和`tf_metric_update`，**下面的代码是错误的示范**

```python
_ , score = session.run([tf_metric_update,tf_metric],\
                        feed_dict=feed_dict)
```

## update_op究竟返回了什么捏

此处参考了
[stackoverflow的一个回答](https://stackoverflow.com/questions/50291916/what-does-the-update-op-return-variable-mean-in-precision-at-k-metric-in-tenso)

具体代码如下

In [12]:
rel = tf.placeholder(tf.int64, [1,3])
rec = tf.constant([[7, 5, 10, 6, 3, 1, 8, 12, 31, 88]], tf.int64) 
precision, update_op = tf.metrics.precision_at_k(rel, rec, 10)

sess = tf.Session()
sess.run(tf.local_variables_initializer())

stream_vars = [i for i in tf.local_variables()]
#Get the local variables true_positive and false_positive

print("[PRECSION_1]: ",sess.run(precision, {rel:[[1,5,10]]})) # nan
#tf.metrics.precision maintains two variables true_positives 
#and  false_positives, each starts at zero.
#so the output at this step is 'nan'

print("[UPDATE_OP_1]:",sess.run(update_op, {rel:[[1,5,10]]})) #0.2
#when the update_op is called, it updates true_positives 
#and false_positives using labels and predictions.

print("[STREAM_VARS_1]:",sess.run(stream_vars)) #[2.0, 8.0]
# Get true positive rate and false positive rate

print("[PRECISION_1]:",sess.run(precision,{rel:[[1,10,15]]})) # 0.2
#So calling precision will use true_positives and false_positives and outputs 0.2

print("[UPDATE_OP_2]:",sess.run(update_op,{rel:[[1,10,15]]})) #0.15
#the update_op updates the values to the new calculated value 0.15.

print("[STREAM_VARS_2]:",sess.run(stream_vars)) #[3.0, 17.0]

[PRECSION_1]:  nan
[UPDATE_OP_1]: 0.2
[STREAM_VARS_1]: [2.0, 8.0]
[PRECISION_1]: 0.2
[UPDATE_OP_2]: 0.15
[STREAM_VARS_2]: [3.0, 17.0]


## tf.metrics.precision_at_k

上面的代码中，我们看到运用的是`tf.metrics.precision_at_k()`这个API，这里的`k`是什么呢？
首先，我们要理解一个概念，究竟什么是`Precision at k`，这里有两份资料，应该能很好地帮助你理解这个概念。
澜子就是看了这两份资料之后，理解了`Precision at k`的概念的。

* [一份PPT](https://ils.unc.edu/courses/2013_spring/inls509_001/lectures/10-EvaluationMetrics.pdf)
* [一篇博客](https://medium.com/@m_n_malaeb/recall-and-precision-at-k-for-recommender-systems-618483226c54)

然后我们来看看这个函数是怎么用的，第一步当然要先看看输入啦。

* [官方API文档在此](https://tensorflow.google.cn/api_docs/python/tf/metrics/precision_at_k)

```python
tf.metrics.precision_at_k(
    labels,
    predictions,
    k,
    class_id=None,
    weights=None,
    metrics_collections=None,
    updates_collections=None,
    name=None
)
```

我们重点关注**labels,predictions,k**这三个参数，应该可以满足日常简单地使用了。
那**labels,predictions,k**的输入形式是什么样的呢？

闲话不说，直接看看上面的栗子。栗子中`rel`其实对应为`labels`，`rec`对应为`predictions`，那`k`又是什么意思呢？
**划重点**：这里的`k`表明你需要对多少个预测样本进行排序。这样说可能有一点抽象，给一个解释。

> Precision@k = (Recommended items @k that are relevant) / (# Recommended items @k)

可以先去看一下
[Github](https://github.com/tensorflow/tensorflow/blob/r1.10/tensorflow/python/ops/metrics_impl.py)，
发现其实在`tf.metrics.precision_at_k`这个函数中，对于`predictions`会根据输入的`k值`进行`top_k`操作。
对应上面的代码中，当`k=10`，即对`rec = tf.constant([[7, 5, 10, 6, 3, 1, 8, 12, 31, 88]], tf.int64)`
所有的样本进行排序，进而在函数中实际运用的是`rec`中**样本数值从大到小排列的索引值**。这样解释应该就能看懂上面代码的意思了。

后来，澜子又在
* [stackoverflow](https://stackoverflow.com/questions/44034262/give-me-a-code-example-usage-for-tensorflows-tf-metrics-sparse-average-precisio/52055189#52055189)
* [知乎](https://www.zhihu.com/question/277184041/answer/480219663)

看到有人问怎么用`tf.metrics.sparse_average_precision_at_k`，就又去求是了一波，
还完成了**知乎的技术首答以及stackoverflow上第一个赞**，
欢迎互粉[知乎](https://www.zhihu.com/people/hong-lan-99/activities)
和[stackoverflow](https://stackoverflow.com/users/8403323/hong-lan)哇。下面给出栗子和简单解释啦。

In [13]:
import tensorflow as tf
import numpy as np

y_true = np.array([[2], [1], [0], [3], [0]]).astype(np.int64)
y_true = tf.identity(y_true)

y_pred = np.array([[0.1, 0.2, 0.6, 0.1],
                   [0.8, 0.05, 0.1, 0.05],
                   [0.3, 0.4, 0.1, 0.2],
                   [0.6, 0.25, 0.1, 0.05],
                   [0.1, 0.2, 0.6, 0.1]
                   ]).astype(np.float32)
y_pred = tf.identity(y_pred)

_, m_ap = tf.metrics.sparse_average_precision_at_k(y_true, y_pred, 3)

sess = tf.Session()
sess.run(tf.local_variables_initializer())

stream_vars = [i for i in tf.local_variables()]

tf_map = sess.run(m_ap)
print("TF_MAP",tf_map)

print("STREAM_VARS",(sess.run(stream_vars)))

tmp_rank = tf.nn.top_k(y_pred,3)

print("TMP_RANK",sess.run(tmp_rank))

Instructions for updating:
Use average_precision_at_k instead
TF_MAP 0.4333333333333333
STREAM_VARS [0.0, 0.0, 5.0, 2.1666666666666665]
TMP_RANK TopKV2(values=array([[0.6 , 0.2 , 0.1 ],
       [0.8 , 0.1 , 0.05],
       [0.4 , 0.3 , 0.2 ],
       [0.6 , 0.25, 0.1 ],
       [0.6 , 0.2 , 0.1 ]], dtype=float32), indices=array([[2, 1, 0],
       [0, 2, 1],
       [1, 0, 3],
       [0, 1, 2],
       [2, 1, 0]], dtype=int32))


## 简单解释一下
* 首先`y_true`代表标签值（未经过**one-hot**），`shape：(batch_size, num_labels)` ,`y_pred`代表预测值（**logit值**） ，`shape：(batch_size, num_classes)`

* 其次，要注意的是`tf.metrics.sparse_average_precision_at_k`中会采用`top_k`根据不同的`k值`对`y_pred`进行排序操作 ，所以`tmp_rank`是为了帮助大噶理解究竟y_pred在函数中进行了怎样的转换。

* 然后，`stream_vars = [i for i in tf.local_variables()]`这一行是为了帮助大噶理解 `tf.metrics.sparse_average_precision_at_k`创建的`tf.local_varibles` 实际输出值，进而可以更好地理解这个函数的用法。

* 具体看这个例子，当`k=1`时，只有第一个**batch的预测输出是和标签匹配的** ，所以最终输出为：`1/6 = 0.166666` ；当`k=2`时，**除了第一个batch的预测输出，第三个batch的预测输出也是和标签匹配的**，所以最终输出为：`(1+(1/2))/6 = 0.25。`

**P.S:在以后的tf版本里，将`tf.metrics.average_precision_at_k`替代`tf.metrics.sparse_average_precision_at_k`**